In [ ]:
import torch
import numpy

Simple residual-conv module

In [ ]:
import torch.nn.functional as F
from torch.nn import Conv2d, ConvTranspose2d

class ResBlk(torch.nn.Module):
    def __init__(self, in_channels=32):
        super().__init__()
        
        n, m = in_channels, in_channels * 2
        self.conv1 = Conv2d(n, m, 3, padding=0)
        self.conv2 = Conv2d(m, m, 1, padding=0)
        self.conv3 = ConvTranspose2d(m, n, 3, padding=0)
    
    def forward(self, input):
        x = F.relu(self.conv1(input))
        x = F.relu(self.conv2(x))
        z = input + self.conv3(x)
        return z 

class ResMod(torch.nn.Module):
    def __init__(self, in_channels=3, out_channels=128):
        super().__init__()

        self.conv1 = Conv2d(3, 32, 1)
        self.rb1 = ResBlk(32)

        self.conv2 = Conv2d(32, 64, 3)
        self.rb2 = ResBlk(64)

        self.conv3 = Conv2d(64, 128, 3)
    
    def forward(self, input):
        x = input
        x = F.relu(self.rb1(self.conv1(x)))
        x = F.relu(self.rb2(self.conv2(x)))
        return self.conv3(x).flatten(-3, -1)

The policy module

In [ ]:
from torch.nn import LSTM, Linear

class RDMModule(torch.nn.Module):
    def __init__(self, n_actions=5):
        super().__init__()
        
        self.features = ResMod()
        self.core = LSTM(128, 128, 1, batch_first=False, bidirectional=False)
        self.output = Linear(128, 1 + n_actions)
    
    def forward(self, input, goal):
        pass

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from gym_discomaze.ext import ExploreRandomDiscoMaze
from gym_discomaze import RandomDiscoMaze

In [ ]:
def factory():
    return ExploreRandomDiscoMaze(5, 5, field=(2, 2))

In [ ]:
env = factory()

In [ ]:
while env.render(mode='human'):
    pass

In [ ]:
env.close()

In [ ]:
torch.randn(2, 3, 4, 2).flatten(1).shape

In [ ]:
def batchify(fn, input, *, end):
    # b_0 x ... x b_{m} x ... ->> [b_0 x ... x b_{m}] x ...
    out = fn(input.flatten(0, end))
    return out.view(input.shape[:end + 1] + out.shape[1:])

In [ ]:
from rlplay.engine.core import context
from rlplay.engine.utils.shared import aliased

ctx = context(env)
buf = aliased(ctx.npy, 16)

In [ ]:
from rlplay.engine.utils.plyr import suply
from rlplay.engine.rollout.same import rollout

In [ ]:
buf.pyt.

In [ ]:
stepno, obs, act, rew, fin = buf.pyt

In [ ]:
x = obs.permute(0, 1, 4, 2, 3).div(255).sub_(0.5)

y = x.flatten(0, 1)

In [ ]:
env.action_space

In [ ]:
res = ResMod()

In [ ]:
res(y).shape

In [ ]:
# out = batchify(conv1, x, end=1)

In [ ]:
out.shape

<br>